# Merging, sorting and joining of data

In [1]:
# Start writing code here...
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Database-style DataFrame or named Series joining/merging
pandas has full-featured, high performance in-memory join operations idiomatically very similar to relational databases like SQL. These methods perform significantly better (in some cases well over an order of magnitude better) than other open source implementations (like base::merge.data.frame in R). The reason for this is careful algorithmic design and the internal layout of the data in DataFrame.


Users who are familiar with SQL but new to pandas might be interested in a comparison with SQL.

pandas provides a single function, merge(), as the entry point for all standard database join operations between DataFrame or named Series objects:  
`pd.merge(
    left,
    right,
    how="inner",
    on=None,
    left_on=None,
    right_on=None,
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)`
* <h5 style="color:red">left:</h5> A DataFrame or named Series object.

* <h5 style="color:red">right:</h5> Another DataFrame or named Series object.

* <h5 style="color:red">on:</h5> Column or index level names to join on. Must be found in both the left and right DataFrame and/or Series objects. If not passed and left_index and right_index are False, the intersection of the columns in the DataFrames and/or Series will be inferred to be the join keys.

* <h5 style="color:red">left_on:</h5> Columns or index levels from the left DataFrame or Series to use as keys. Can either be column names, index level names, or arrays with length equal to the length of the DataFrame or Series.

* <h5 style="color:red">right_on:</h5> Columns or index levels from the right DataFrame or Series to use as keys. Can either be column names, index level names, or arrays with length equal to the length of the DataFrame or Series.

* <h5 style="color:red">left_index:</h5> If True, use the index (row labels) from the left DataFrame or Series as its join key(s). In the case of a DataFrame or Series with a MultiIndex (hierarchical), the number of levels must match the number of join keys from the right DataFrame or Series.

* <h5 style="color:red">right_index:</h5> Same usage as left_index for the right DataFrame or Series

* <h5 style="color:red">how:</h5> One of 'left', 'right', 'outer', 'inner'. Defaults to inner. See below for more detailed description of each method.

* <h5 style="color:red">sort:</h5> Sort the result DataFrame by the join keys in lexicographical order. Defaults to True, setting to False will improve performance substantially in many cases.

* <h5 style="color:red">suffixes:</h5> A tuple of string suffixes to apply to overlapping columns. Defaults to ('_x', '_y').

* <h5 style="color:red">copy:</h5> Always copy data (default True) from the passed DataFrame or named Series objects, even when reindexing is not necessary. Cannot be avoided in many cases but may improve performance / memory usage. The cases where copying can be avoided are somewhat pathological but this option is provided nonetheless.

* <h5 style="color:red">indicator:</h5> Add a column to the output DataFrame called _merge with information on the source of each row. _merge is Categorical-type and takes on a value of left_only for observations whose merge key only appears in 'left' DataFrame or Series, right_only for observations whose merge key only appears in 'right' DataFrame or Series, and both if the observation’s merge key is found in both.

* <h5 style="color:red">validate :</h5> string, default None. If specified, checks if merge is of specified type.

    * “one_to_one” or “1:1”: checks if merge keys are unique in both left and right datasets.

    * “one_to_many” or “1:m”: checks if merge keys are unique in left dataset.

    * “many_to_one” or “m:1”: checks if merge keys are unique in right dataset.

    * “many_to_many” or “m:m”: allowed, but does not result in checks.
`Note:`Support for specifying index levels as the on, left_on, and right_on parameters was added in version 0.23.0. Support for merging named Series objects was added in version 0.24.0.

The return type will be the same as left. If left is a DataFrame or named Series and right is a subclass of DataFrame, the return type will still be DataFrame.

merge is a function in the pandas namespace, and it is also available as a DataFrame instance method merge(), with the calling DataFrame being implicitly considered the left object in the join.

The related join() method, uses merge internally for the index-on-index (by default) and column(s)-on-index join. If you are joining on index only, you may wish to use DataFrame.join to save yourself some typing.

# Brief primer on merge methods (relational algebra)
Experienced users of relational databases like SQL will be familiar with the terminology used to describe join operations between two SQL-table like structures (DataFrame objects). There are several cases to consider which are very important to understand:

`one-to-one` joins: for example when joining two DataFrame objects on their indexes (which must contain unique values).

`many-to-one` joins: for example when joining an index (unique) to one or more columns in a different DataFrame.

`many-to-many` joins: joining columns on columns.

Note:

When joining columns on columns (potentially a many-to-many join), any indexes on the passed DataFrame objects will be discarded.

It is worth spending some time understanding the result of the many-to-many join case. In SQL / standard relational algebra, if a key combination appears more than once in both tables, the resulting table will have the Cartesian product of the associated data. Here is a very basic example with one unique key combination:

In [2]:
left = pd.DataFrame({
   'id':[1,2,3,4,5],
   'Name': ['Alex', 'Amy', 'Allen', 'Alice', 'Ayoung'],
   'subject_id':['sub1','sub2','sub4','sub6','sub5']})
right = pd.DataFrame(
   {'id':[1,2,3,4,5],
   'Name': ['Billy', 'Brian', 'Bran', 'Bryce', 'Betty'],
   'subject_id':['sub2','sub4','sub3','sub6','sub5']})
print( left )
print( right )

   id    Name subject_id
0   1    Alex       sub1
1   2     Amy       sub2
2   3   Allen       sub4
3   4   Alice       sub6
4   5  Ayoung       sub5
   id   Name subject_id
0   1  Billy       sub2
1   2  Brian       sub4
2   3   Bran       sub3
3   4  Bryce       sub6
4   5  Betty       sub5


Merge Two DataFrames on a Key

In [3]:
print( pd.merge(left,right,on='id'))

   id  Name_x subject_id_x Name_y subject_id_y
0   1    Alex         sub1  Billy         sub2
1   2     Amy         sub2  Brian         sub4
2   3   Allen         sub4   Bran         sub3
3   4   Alice         sub6  Bryce         sub6
4   5  Ayoung         sub5  Betty         sub5


Merge Two DataFrames on Multiple Keys

In [4]:
print( pd.merge(left,right,on=['id','subject_id']))

   id  Name_x subject_id Name_y
0   4   Alice       sub6  Bryce
1   5  Ayoung       sub5  Betty


In [5]:
print (pd.merge(left, right, left_on='subject_id',right_on="subject_id"))

   id_x  Name_x subject_id  id_y Name_y
0     2     Amy       sub2     1  Billy
1     3   Allen       sub4     2  Brian
2     4   Alice       sub6     4  Bryce
3     5  Ayoung       sub5     5  Betty


In [6]:
print (pd.merge(left, right,left_index=True,right_index=True))

   id_x  Name_x subject_id_x  id_y Name_y subject_id_y
0     1    Alex         sub1     1  Billy         sub2
1     2     Amy         sub2     2  Brian         sub4
2     3   Allen         sub4     3   Bran         sub3
3     4   Alice         sub6     4  Bryce         sub6
4     5  Ayoung         sub5     5  Betty         sub5


In [7]:
print (pd.merge(left, right,left_index=True,right_on="id"))

   id  id_x  Name_x subject_id_x  id_y Name_y subject_id_y
0   1     2     Amy         sub2     1  Billy         sub2
1   2     3   Allen         sub4     2  Brian         sub4
2   3     4   Alice         sub6     3   Bran         sub3
3   4     5  Ayoung         sub5     4  Bryce         sub6


In [8]:
print (pd.merge(left, right,on="id",sort=True))

   id  Name_x subject_id_x Name_y subject_id_y
0   1    Alex         sub1  Billy         sub2
1   2     Amy         sub2  Brian         sub4
2   3   Allen         sub4   Bran         sub3
3   4   Alice         sub6  Bryce         sub6
4   5  Ayoung         sub5  Betty         sub5


In [9]:
left_ = pd.DataFrame({
   'id':[1,2,3,4,5],
   'Name': ['Alex', 'Amy', 'Allen', 'Alice', 'Ayoung'],
   'subject_id':['sub1','sub3','sub2','sub4','sub6']})
right_ = pd.DataFrame(
   {'id':[1,2,3,4,5],
   'Name': ['Billy', 'Brian', 'Bran', 'Bryce', 'Betty'],
   'subject_id':['sub3','sub2','sub1','sub6','sub5']})

In [10]:
print (pd.merge(left_, right_,on="subject_id",sort=True)) #with sort

   id_x  Name_x subject_id  id_y Name_y
0     1    Alex       sub1     3   Bran
1     3   Allen       sub2     2  Brian
2     2     Amy       sub3     1  Billy
3     5  Ayoung       sub6     4  Bryce


In [11]:
print (pd.merge(left_, right_,on="subject_id"))# with out sort

   id_x  Name_x subject_id  id_y Name_y
0     1    Alex       sub1     3   Bran
1     2     Amy       sub3     1  Billy
2     3   Allen       sub2     2  Brian
3     5  Ayoung       sub6     4  Bryce


In [12]:
print (pd.merge(left, right,left_index=True,right_on="id",suffixes=["_left","_right"])) # by changing suffixes

   id  id_left Name_left subject_id_left  id_right Name_right subject_id_right
0   1        2       Amy            sub2         1      Billy             sub2
1   2        3     Allen            sub4         2      Brian             sub4
2   3        4     Alice            sub6         3       Bran             sub3
3   4        5    Ayoung            sub5         4      Bryce             sub6


### Merge Using 'how' Argument
The how argument to merge specifies how to determine which keys are to be included in the resulting table. If a key combination does not appear in either the left or the right tables, the values in the joined table will be NA.

Here is a summary of the how options and their SQL equivalent names −

|Merge Method|	SQL Equivalent|	Description|
|:---:|:---|---:|
|left|	LEFT OUTER JOIN	|Use keys from left object|
|right|	RIGHT OUTER JOIN|	Use keys from right object|
|outer|	FULL OUTER JOIN|	Use union of keys|
|inner|	INNER JOIN|	Use intersection of keys|


### Left Join

In [13]:
#Left Join
print (pd.merge(left, right, on='subject_id', how='left'))

   id_x  Name_x subject_id  id_y Name_y
0     1    Alex       sub1   NaN    NaN
1     2     Amy       sub2   1.0  Billy
2     3   Allen       sub4   2.0  Brian
3     4   Alice       sub6   4.0  Bryce
4     5  Ayoung       sub5   5.0  Betty


### Right Join

In [14]:
print (pd.merge(left, right, on='subject_id', how='right'))

   id_x  Name_x subject_id  id_y Name_y
0   2.0     Amy       sub2     1  Billy
1   3.0   Allen       sub4     2  Brian
2   NaN     NaN       sub3     3   Bran
3   4.0   Alice       sub6     4  Bryce
4   5.0  Ayoung       sub5     5  Betty


### Outer Join

In [15]:
print (pd.merge(left, right, how='outer', on='subject_id'))

   id_x  Name_x subject_id  id_y Name_y
0   1.0    Alex       sub1   NaN    NaN
1   2.0     Amy       sub2   1.0  Billy
2   3.0   Allen       sub4   2.0  Brian
3   4.0   Alice       sub6   4.0  Bryce
4   5.0  Ayoung       sub5   5.0  Betty
5   NaN     NaN       sub3   3.0   Bran


### Inner Join

In [16]:
print( pd.merge(left, right, on='subject_id', how='inner'))

   id_x  Name_x subject_id  id_y Name_y
0     2     Amy       sub2     1  Billy
1     3   Allen       sub4     2  Brian
2     4   Alice       sub6     4  Bryce
3     5  Ayoung       sub5     5  Betty


In [17]:
s_abb=pd.read_csv("https://raw.githubusercontent.com/reddyprasade/DataSet-for-ML-and-Data-Science/master/DataSets/state-abbrevs.csv")
s_area=pd.read_csv("https://raw.githubusercontent.com/reddyprasade/DataSet-for-ML-and-Data-Science/master/DataSets/state-areas.csv")
s_pop=pd.read_csv("https://raw.githubusercontent.com/reddyprasade/DataSet-for-ML-and-Data-Science/master/DataSets/state-population.csv")

In [18]:
display(s_area.head(),s_abb.head(),s_pop.head())

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


In [19]:
s_abb.isnull().sum()

state           0
abbreviation    0
dtype: int64

In [20]:
s_abb.abbreviation.unique()

array(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA',
       'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MT', 'NE',
       'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'MD',
       'MA', 'MI', 'MN', 'MS', 'MO', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'], dtype=object)

In [21]:
s_area.isnull().sum()

In [22]:
s_area.state.unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
       'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
       'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma',
       'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
       'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming',
       'District of Columbia', 'Puerto Rico'], dtype=object)

In [23]:
s_pop.isnull().sum()

state/region     0
ages             0
year             0
population      20
dtype: int64

In [24]:
s_pop[s_pop['population'].isnull()] #??

,state/region,ages,year,population
2448,PR,under18,1990,NaN
2449,PR,total,1990,NaN
2450,PR,total,1991,NaN
2451,PR,under18,1991,NaN
2452,PR,total,1993,NaN
2453,PR,under18,1993,NaN
2454,PR,under18,1992,NaN
2455,PR,total,1992,NaN
2456,PR,under18,1994,NaN
2457,PR,total,1994,NaN


In [25]:
merged=pd.merge(s_pop,s_abb,how="outer",left_on="state/region",right_on="abbreviation")

In [26]:
merged

,state/region,ages,year,population,state,abbreviation
0,AL,under18,2012,1117489.0,Alabama,AL
1,AL,total,2012,4817528.0,Alabama,AL
2,AL,under18,2010,1130966.0,Alabama,AL
3,AL,total,2010,4785570.0,Alabama,AL
4,AL,under18,2011,1125763.0,Alabama,AL
...,...,...,...,...,...,...
2539,USA,total,2010,309326295.0,NaN,NaN
2540,USA,under18,2011,73902222.0,NaN,NaN
2541,USA,total,2011,311582564.0,NaN,NaN
2542,USA,under18,2012,73708179.0,NaN,NaN


In [35]:
merged.drop('abbreviation',axis=1,inplace=True)

In [36]:
merged.isnull().sum()

state/region     0
ages             0
year             0
population      20
state            0
dtype: int64

In [37]:
s_area.state.unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
       'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
       'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma',
       'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
       'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming',
       'District of Columbia', 'Puerto Rico'], dtype=object)

In [38]:
merged.loc[merged['state'].isnull(),"state/region"].unique()

array([], dtype=object)

In [39]:
merged.loc[merged['state/region']=="PR","state"]="Puerto Rico"

In [40]:
merged.loc[merged['state/region']=="USA","state"]="United States of America"

In [41]:
merged.isnull().sum() # since we can not fill population data we leave that for now 

state/region     0
ages             0
year             0
population      20
state            0
dtype: int64

In [43]:
final_data=pd.merge(merged,s_area,how="outer")

In [44]:
final_data.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [48]:
final_data.isnull().sum()

state/region      0
ages              0
year              0
population       20
state             0
area (sq. mi)    48
dtype: int64

In [49]:
final_data['state'][final_data["area (sq. mi)"].isnull()].unique()

array(['United States of America'], dtype=object)

 since we can not fill population and area data we decided to drop that data

In [52]:
final_data.dropna(inplace=True)

In [53]:
final_data.isnull().sum()

state/region     0
ages             0
year             0
population       0
state            0
area (sq. mi)    0
dtype: int64

Here as we can see there are no null values in this data and it is ready to perform furtther analytics or to export 

In [54]:
final_data.to_csv("/work/Data-Analysis-with-Python/Data/USA_final_data.csv'")

Queries

In [59]:
data2010=final_data.query("year==2010 & ages=='total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


In [63]:
data2010.set_index("state",inplace=True)

In [64]:
density=data2010['population']/data2010['area (sq. mi)']

In [65]:
density.head()

state
Alabama        91.287603
Alaska          1.087509
Arizona        56.214497
Arkansas       54.948667
California    228.051342
dtype: float64

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=fcf2399d-084b-4173-af36-20a4a45218a8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>